In [1]:
import model

In [2]:
# u = readCsv('disorder.csv')

In [3]:
firstModel = model.IoisModel(verbose = True)

In [4]:
%prun firstModel.run_model()

['Time: ', 1, 'Apps: ', 41, 'Pay: ', 2558.535338864839, 'Cost', 2460.0, 'Actor Connections', 616, 'Application Connections', 2508]
['Time: ', 2, 'Apps: ', 41, 'Pay: ', 5989.598336639995, 'Cost', 2579.0, 'Actor Connections', 610, 'Application Connections', 3087]
['Time: ', 3, 'Apps: ', 41, 'Pay: ', 9437.05058789437, 'Cost', 2819.0, 'Actor Connections', 637, 'Application Connections', 3602]
['Time: ', 4, 'Apps: ', 41, 'Pay: ', 12916.653701045567, 'Cost', 3179.0, 'Actor Connections', 672, 'Application Connections', 4022]
['Time: ', 5, 'Apps: ', 42, 'Pay: ', 16418.969622133227, 'Cost', 3715.0, 'Actor Connections', 691, 'Application Connections', 4328]
['Time: ', 6, 'Apps: ', 42, 'Pay: ', 19595.559166155577, 'Cost', 4315.0, 'Actor Connections', 713, 'Application Connections', 4657]
['Time: ', 7, 'Apps: ', 42, 'Pay: ', 23088.660918539324, 'Cost', 5035.0, 'Actor Connections', 745, 'Application Connections', 4701]
['Time: ', 8, 'Apps: ', 42, 'Pay: ', 26565.010154837553, 'Cost', 5875.0, 'Actor 

In [10]:
for app in firstModel.schedule.appList.values():
    print(app.av)

('1', '0', '?', '0', '?', '?', '0', '0', '0', '0')
('?', '0', '0', '?', '0', '0', '?', '0', '0', '0')
('0', '1', '0', '?', '1', '?', '?', '0', '1', '0')
('0', '?', '0', '0', '1', '0', '0', '?', '1', '?')
('?', '0', '?', '0', '0', '1', '0', '1', '?', '1')
('?', '0', '?', '1', '?', '1', '1', '0', '0', '1')
('0', '?', '?', '0', '0', '0', '?', '0', '1', '1')
('0', '1', '?', '?', '0', '0', '?', '1', '0', '1')
('0', '0', '?', '?', '0', '0', '0', '0', '?', '1')
('0', '0', '1', '?', '1', '1', '0', '0', '?', '?')
('0', '0', '1', '1', '0', '0', '?', '1', '?', '?')
('0', '1', '1', '0', '0', '?', '?', '?', '0', '0')
('0', '?', '0', '?', '1', '0', '1', '0', '1', '?')
('1', '0', '?', '1', '0', '1', '1', '?', '1', '?')
('1', '?', '?', '1', '1', '?', '0', '0', '0', '1')
('0', '?', '?', '0', '0', '0', '0', '1', '0', '?')
('?', '0', '1', '?', '1', '0', '0', '0', '0', '?')


In [6]:
import time
import numpy as np

In [7]:
vector = firstModel.schedule.allOpportunities[1].opportunities[0].vector

In [12]:
np.fromiter((abs(i - 1) if np.random.binomial(1, firstModel.ambiguity) == 1 else i for i in vector),int)

array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0])

In [49]:
np.fromiter((abs(i - 1) if np.random.binomial(1, firstModel.ambiguity) == 1 else i for i in np.copy(vector)),dtype='U1')

array(['0', '1', '0', '0', '0', '0', '1', '1', '0', '0'], dtype='<U1')

In [11]:
np.array(vector,int)

array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0])

In [80]:
from mesa import Agent
import numpy as np
from utility import match
from operator import itemgetter

def communicate1(self, schedule):
        """
        The agent communicates
        """
        edgesToAdd = []
        for s,actorKey,d in schedule.network.edges(self.unique_id, data=True):
            if actorKey in schedule.actorList and schedule.actorList[actorKey].tier != self.tier and d['strength'] >= self.model.commThreshold:

                d['strength'] = min(d['strength'] + self.model.commGain, 1)
                
                for app in list(set(schedule.network.neighbors(actorKey)) | set(schedule.network.neighbors(s))):
                    if app not in schedule.actorList:
                        try:
                            ownKnow = schedule.network[self.unique_id][app]['strength']
                        except KeyError:
                            edgesToAdd.append((self.unique_id, app, schedule.network[actorKey][app]['strength'] * self.model.commDensity))
                            continue
                        try:
                            otherKnow = schedule.network[actorKey][app]['strength']
                        except KeyError:
                            edgesToAdd.append((actorKey, app, ownKnow * self.model.commDensity))
                            continue
                        diff = ownKnow - otherKnow
                        if diff > 0:
                            schedule.network[actorKey][app]['strength'] += diff * self.model.commDensity
                        elif diff < 0:
                            schedule.network[self.unique_id][app]['strength'] += -diff * self.model.commDensity
                # as array: (app, node1, node2) aber dann keine Verbindung zu nodes
                
        schedule.network.add_weighted_edges_from(edgesToAdd, 'strength')


def communicate2(self, schedule):
        """
        The agent communicates
        """
        edgesToAdd = []
        for s,actorKey,d in schedule.network.edges(self.unique_id, data=True):
            if actorKey in schedule.actorList and schedule.actorList[actorKey].tier != self.tier and d['strength'] >= self.model.commThreshold:

                d['strength'] = min(d['strength'] + self.model.commGain, 1)
                
                for app in list(set(schedule.network.neighbors(actorKey)) | set(schedule.network.neighbors(s))):
                    if app not in schedule.actorList:
                        try:
                            ownKnow = schedule.network[self.unique_id][app]['strength']
                        except KeyError:
                            edgesToAdd.append((self.unique_id, app, schedule.network[actorKey][app]['strength'] * self.model.commDensity))
                            continue
                        try:
                            otherKnow = schedule.network[actorKey][app]['strength']
                        except KeyError:
                            edgesToAdd.append((actorKey, app, ownKnow * self.model.commDensity))
                            continue
                        diff = ownKnow - otherKnow
                        if diff > 0:
                            schedule.network[actorKey][app]['strength'] += diff * self.model.commDensity
                        elif diff < 0:
                            schedule.network[self.unique_id][app]['strength'] += -diff * self.model.commDensity
                # as array: (app, node1, node2) aber dann keine Verbindung zu nodes
                
        schedule.network.add_weighted_edges_from(edgesToAdd, 'strength')

In [8]:
from utility import match
import time
import networkx as nx
import pandas as pd
import numpy as np

In [82]:
self = firstModel.schedule.actorList[0]
perceived=[0,1,0,1,0,1,0,1,0,1]
schedule=firstModel.schedule
time1 = 0.0
for i in range(1000):
    start1 = time.time()
    communicate1(self, schedule)
    stop1 = time.time()
    time1 += stop1 - start1

self = firstModel.schedule.actorList[0]
perceived=[0,1,0,1,0,1,0,1,0,1]
schedule=firstModel.schedule
time2 = 0.0
for i in range(1000):
    start2 = time.time()
    communicate2(self, schedule)
    stop2 = time.time()
    time2 += stop2 - start2

print(time1)
print(time2)

2.331467628479004
56.53080701828003


In [28]:
v1 = firstModel.schedule.appList[120].av
v2 = (0,1,0,1,0,1,0,1,0,1)

In [5]:
firstModel.schedule.appList

{120: <agents.Application at 0x2a018ad2550>,
 121: <agents.Application at 0x2a018ad24a8>,
 122: <agents.Application at 0x2a056239320>,
 123: <agents.Application at 0x2a018ad23c8>,
 124: <agents.Application at 0x2a018ada198>,
 125: <agents.Application at 0x2a018ada240>,
 126: <agents.Application at 0x2a018ada2e8>,
 127: <agents.Application at 0x2a018ada390>,
 128: <agents.Application at 0x2a018ada438>,
 129: <agents.Application at 0x2a018ada4e0>,
 130: <agents.Application at 0x2a018ada588>,
 131: <agents.Application at 0x2a018ada630>,
 132: <agents.Application at 0x2a018ada6d8>,
 133: <agents.Application at 0x2a018ada780>,
 134: <agents.Application at 0x2a018ada828>,
 135: <agents.Application at 0x2a018ada8d0>,
 136: <agents.Application at 0x2a018ada978>,
 137: <agents.Application at 0x2a018adaa20>,
 138: <agents.Application at 0x2a018adaac8>,
 139: <agents.Application at 0x2a018adab70>,
 140: <agents.Application at 0x2a018adac18>,
 141: <agents.Application at 0x2a018adacc0>,
 142: <age

In [31]:
def sum1(v1, v2):
    sum = 0.0
    for a,b in zip(v1, v2):
        if a == '?':
            sum += 0.5
        elif int(a) == b:
            sum += 1.0
    return sum

def sum2(v1, v2):
    sum = 0.0
    for a,b in zip(v1, v2):
        try:
            if int(a) == b: sum += 1.0
        except ValueError:
            sum += 0.5
    return sum

In [32]:
print(sum1(v2, v2))

10.0


In [25]:
time1 = 0.0
for i in range(1000000):
    start1 = time.time()
    sum = sum1(v1, v2)
    stop1 = time.time()
    time1 += stop1 - start1

time2 = 0.0
for i in range(1000000):
    start2 = time.time()
    sum = sum2(v1, v2)
    stop2 = time.time()
    time2 += stop2 - start2

print(time1)
print(time2)

3.5280344486236572
9.025083780288696


In [11]:
import time
import numpy as np
from random import choices
from random import choice

In [44]:
schedule = firstModel.schedule
self = firstModel.schedule

In [1]:
from experiment import runExperiment
from multiprocessing import Pool
import csv

In [2]:
#10 runs in 271.2s
#for range in range(50,61):
#    print(runExperiment(range))

In [3]:
#10 runs in 109.5s
#1000 runs in 
results = set()
for i in range(2):
    p = Pool(8)
    results = results.union(p.map(runExperiment, [round(x * 0.01, 2) for x in range(101)]))
    p.close()
    p.join()
    print(f'Iteration {i + 1} finished.')

with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
    for r in results:
        writer.writerow(r)

Iteration 1 finished.
Iteration 2 finished.


In [5]:
with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
    for r in results:
        writer.writerow(r)